# Problem 3

##### Explore and cluster the neighborhoods in Toronto.
##### Generate maps to visualize your neighborhoods and how they cluster together

In [10]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import os
from sklearn.cluster import KMeans

from geopy.geocoders import Nominatim 
import matplotlib.cm as cm
import matplotlib.colors as colors


print('Libraries imported.')

Libraries imported.


In [11]:
List_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(List_url).text

In [12]:
soup = BeautifulSoup(source, 'xml')

In [13]:
table=soup.find('table')

In [14]:
column_names=['Postalcode','Borough','Neighbourhood']
df = pd.DataFrame(columns=column_names)

In [15]:
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data

In [16]:
df.head()

,Postalcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [17]:
df=df[df['Borough']!='Not assigned']

In [18]:
df[df['Neighbourhood']=='Not assigned']=df['Borough']

ValueError: cannot set using a list-like indexer with a different length than the value

In [19]:
df.head()

,Postalcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [20]:
temp_df=df.groupby('Postalcode')['Neighbourhood'].apply(lambda x: "%s" % ', '.join(x))
temp_df=temp_df.reset_index(drop=False)
temp_df.rename(columns={'Neighbourhood':'Neighbourhood_joined'},inplace=True)

In [21]:
df_merge = pd.merge(df, temp_df, on='Postalcode')

In [22]:
df_merge.drop(['Neighbourhood'],axis=1,inplace=True)

In [23]:
df_merge.drop_duplicates(inplace=True)

In [24]:
df_merge.rename(columns={'Neighbourhood_joined':'Neighbourhood'},inplace=True)

In [25]:
df_merge.head()

,Postalcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [111]:
df_merge.shape

(103, 3)

In [27]:
def get_geocode(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude,longitude

In [28]:
geo_df=pd.read_csv('http://cocl.us/Geospatial_data')

In [29]:
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [30]:
geo_df.rename(columns={'Postal Code':'Postalcode'},inplace=True)
geo_merged = pd.merge(geo_df, df_merge, on='Postalcode')

In [31]:
geo_data=geo_merged[['Postalcode','Borough','Neighbourhood','Latitude','Longitude']]

In [32]:
geo_data.head()

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [33]:
toronto_data=geo_data[geo_data['Borough'].str.contains("Toronto")]
toronto_data.head()

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [128]:
CLIENT_ID = 'QEF4SFROUUVOQKNPAFNNRDUW4ACAWSYYG312LM3BKEDDKPIZ'
CLIENT_SECRET = 'CCBFW1WDVTUYDLPFX3SR0QBB5R2UKRYOZ3F1JDJ3PLQMPDRM' 
VERSION = '20180604'

In [10]:
def getNearbyVenues(names, latitudes, longitudes):
    radius=500
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
         # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
 
    # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        
        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
   

In [16]:
geo_df=pd.read_csv('http://cocl.us/Geospatial_data')

In [20]:
column_names=['Postalcode','Borough','Neighborhood']

In [21]:
df = pd.DataFrame(columns=column_names)

In [32]:
temp_df=df.groupby('Postalcode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
temp_df=temp_df.reset_index(drop=False)
temp_df.rename(columns={'Neighborhood':'Neighborhood_joined'},inplace=True)

In [33]:
df_merge = pd.merge(df, temp_df, on='Postalcode')

In [35]:
geo_df.rename(columns={'Postal Code':'Postalcode'},inplace=True)
geo_merged = pd.merge(geo_df, df_merge, on='Postalcode')

In [36]:
geo_data=geo_merged[['Postalcode','Borough','Neighborhood','Latitude','Longitude']]
geo_data.head()


,Postalcode,Borough,Neighborhood,Latitude,Longitude


In [37]:
toronto_data=geo_data[geo_data['Borough'].str.contains("Toronto")]
toronto_data.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude


In [39]:
def getNearbyVenues(names, latitudes, longitudes):
    radius=500
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
         # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
                
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
 
    # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        
        
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)
    

In [ ]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The Junction South
Parkdale, Roncesvalles
Runnymede, Swansea
Business Reply Mail Processing Centre 969 Eastern

In [ ]:
toronto_venues.head()


Neighborhood	Neighborhood Latitude	Neighborhood Longitude	Venue	Venue Latitude	Venue Longitude	Venue Category
0	The Beaches	43.676357	-79.293031	Glen Manor Ravine	43.676821	-79.293942	Trail
1	The Beaches	43.676357	-79.293031	The Big Carrot Natural Food Market	43.678879	-79.297734	Health Food Store
2	The Beaches	43.676357	-79.293031	Grover Pub and Grub	43.679181	-79.297215	Pub
3	The Beaches	43.676357	-79.293031	Upper Beaches	43.680563	-79.292869	Neighborhood
4	The Danforth West, Riverdale	43.679557	-79.352188	Pantheon	43.677621	-79.351434	Greek Restaurant

In [ ]:
toronto_venues.groupby('Neighborhood').count()

Neighborhood Latitude	Neighborhood Longitude	Venue	Venue Latitude	Venue Longitude	Venue Category
Neighbourhood						
Adelaide, King, Richmond	100	100	100	100	100	100
Agincourt	4	4	4	4	4	4
Agincourt North, L'Amoreaux East, Milliken, Steeles East	2	2	2	2	2	2
Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown	10	10	10	10	10	10
Alderwood, Long Branch	10	10	10	10	10	10
Bathurst Manor, Downsview North, Wilson Heights	18	18	18	18	18	18
Bayview Village	4	4	4	4	4	4
Bedford Park, Lawrence Manor East	24	24	24	24	24	24
Berczy Park	55	55	55	55	55	55
Birch Cliff, Cliffside West	4	4	4	4	4	4
Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe	7	7	7	7	7	7
Brockton, Exhibition Place, Parkdale Village	22	22	22	22	22	22
Business Reply Mail Processing Centre 969 Eastern	17	17	17	17	17	17
CFB Toronto, Downsview East	3	3	3	3	3	3
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara	14	14	14	14	14	14
Cabbagetown, St. James Town	46	46	46	46	46	46
Caledonia-Fairbanks	6	6	6	6	6	6
Canada Post Gateway Processing Centre	11	11	11	11	11	11
Cedarbrae	7	7	7	7	7	7
Central Bay Street	83	83	83	83	83	83
Chinatown, Grange Park, Kensington Market	100	100	100	100	100	100
Christie	16	16	16	16	16	16
Church and Wellesley	86	86	86	86	86	86
Clairlea, Golden Mile, Oakridge	9	9	9	9	9	9
Clarks Corners, Sullivan, Tam O'Shanter	9	9	9	9	9	9
Cliffcrest, Cliffside, Scarborough Village West	3	3	3	3	3	3
Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park	2	2	2	2	2	2
Commerce Court, Victoria Hotel	100	100	100	100	100	100
Davisville	37	37	37	37	37	37
Davisville North	8	8	8	8	8	8
...	...	...	...	...	...	...
North Toronto West	20	20	20	20	20	20
Northwest	3	3	3	3	3	3
Northwood Park, York University	4	4	4	4	4	4
Parkdale, Roncesvalles	16	16	16	16	16	16
Parkwoods	3	3	3	3	3	3
Queen's Park	40	40	40	40	40	40
Rosedale	4	4	4	4	4	4
Roselawn	1	1	1	1	1	1
Rouge, Malvern	1	1	1	1	1	1
Runnymede, Swansea	37	37	37	37	37	37
Ryerson, Garden District	100	100	100	100	100	100
Scarborough Village	1	1	1	1	1
St. James Town	100	100	100	100	100	100
Stn A PO Boxes 25 The Esplanade	94	94	94	94	94	94
Studio District	39	39	39	39	39	39
The Annex, North Midtown, Yorkville	24	24	24	24	24	24
The Beaches	5	5	5	5	5	5
The Beaches West, India Bazaar	21	21	21	21	21	21
The Danforth West, Riverdale	42	42	42	42	42	42
The Junction North, Runnymede	4	4	4	4	4	4
The Kingsway, Montgomery Road, Old Mill North	4	4	4	4	4	4
Thorncliffe Park	17	17	17	17	17	17
Victoria Village	4	4	4	4	4	4
Westmount	7	7	7	7	7	7
Willowdale South	35	35	35	35	35	35
Willowdale West	4	4	4	4	4	4
Woburn	3	3	3	3	3	3
Woodbine Gardens, Parkview Hill	14	14	14	14	14	14
Woodbine Heights	10	10	10	10	10	10
York Mills West	3	3	3	3	3	3

98 rows x 6 columns

How many categorys can we find?

In [ ]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

	Neighbourhood	Accessories Store	Adult Boutique	Afghan Restaurant	Airport	Airport Food Court	Airport Gate	Airport Lounge	Airport Service	Airport Terminal	...	Turkish Restaurant	Vegetarian / Vegan Restaurant	Video Game Store	Video Store	Vietnamese Restaurant	Warehouse Store	Wine Bar	Wings Joint	Women's Store	Yoga Studio
0	Rouge, Malvern	0	0	0	0	0	0	0	0	0	...	0	0	0	0	0	0	0	0	0	0
1	Highland Creek, Rouge Hill, Port Union	0	0	0	0	0	0	0	0	0	...	0	0	0	0	0	0	0	0	0	0
2	Guildwood, Morningside, West Hill	0	0	0	0	0	0	0	0	0	...	0	0	0	0	0	0	0	0	0	0
3	Guildwood, Morningside, West Hill	0	0	0	0	0	0	0	0	0	...	0	0	0	0	0	0	0	0	0	0
4	Guildwood, Morningside, West Hill	0	0	0	0	0	0	0	0	0	...	0	0	0	0	0	0	0	0	0	0

5 rows × 274 columns

In [ ]:
toronto_onehot.shape

(2235, 274)

In [ ]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.head()

	Neighbourhood	Accessories Store	Adult Boutique	Afghan Restaurant	Airport	Airport Food Court	Airport Gate	Airport Lounge	Airport Service	Airport Terminal	...	Turkish Restaurant	Vegetarian / Vegan Restaurant	Video Game Store	Video Store	Vietnamese Restaurant	Warehouse Store	Wine Bar	Wings Joint	Women's Store	Yoga Studio
0	Adelaide, King, Richmond	0.01	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	...	0.0	0.01	0.0	0.0	0.0	0.0	0.01	0.0	0.01	0.0
1	Agincourt	0.00	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	...	0.0	0.00	0.0	0.0	0.0	0.0	0.00	0.0	0.00	0.0
2	Agincourt North, L'Amoreaux East, Milliken, St...	0.00	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	...	0.0	0.00	0.0	0.0	0.0	0.0	0.00	0.0	0.00	0.0
3	Albion Gardens, Beaumond Heights, Humbergate, ...	0.00	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	...	0.0	0.00	0.0	0.0	0.0	0.0	0.00	0.0	0.00	0.0
4	Alderwood, Long Branch	0.00	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	...	0.0	0.00	0.0	0.0	0.0	0.0	0.00	0.0	0.00	0.0

5 rows × 274 columns

In [ ]:
toronto_grouped.shape

(98, 274)

Get the top 10 for each neighbourhood

In [48]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Neighbourhood	1st Most Common Venue	2nd Most Common Venue	3rd Most Common Venue	4th Most Common Venue	5th Most Common Venue	6th Most Common Venue	7th Most Common Venue	8th Most Common Venue	9th Most Common Venue	10th Most Common Venue
0	Adelaide, King, Richmond	Coffee Shop	Café	Steakhouse	Bar	Thai Restaurant	Burger Joint	Gym	Asian Restaurant	Hotel	Bakery
1	Agincourt	Lounge	Sandwich Place	Breakfast Spot	Skating Rink	Dumpling Restaurant	Dog Run	Doner Restaurant	Donut Shop	Drugstore	Electronics Store
2	Agincourt North, L'Amoreaux East, Milliken, St...	Playground	Park	Yoga Studio	Drugstore	Dim Sum Restaurant	Diner	Discount Store	Dog Run	Doner Restaurant	Donut Shop
3	Albion Gardens, Beaumond Heights, Humbergate, ...	Grocery Store	Pizza Place	Fried Chicken Joint	Coffee Shop	Sandwich Place	Fast Food Restaurant	Beer Store	Liquor Store	Pharmacy	Dance Studio
4	Alderwood, Long Branch	Pizza Place	Skating Rink	Dance Studio	Pharmacy	Coffee Shop	Pool	Pub	Sandwich Place	Gym	Airport Terminal


Custer neighbourhoods

In [ ]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0], dtype=int32)
                                                                             Merge the dataframe with the top 10 and the cluster for each neighbourhood

In [ ]:
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

	PostalCode	Borough	Neighborhood	Latitude	Longitude	Cluster Labels	1st Most Common Venue	2nd Most Common Venue	3rd Most Common Venue	4th Most Common Venue	5th Most Common Venue	6th Most Common Venue	7th Most Common Venue	8th Most Common Venue	9th Most Common Venue	10th Most Common Venue
0	M1B	Scarborough	Rouge, Malvern	43.806686	-79.194353	0.0	Fast Food Restaurant	Dumpling Restaurant	Diner	Discount Store	Dog Run	Doner Restaurant	Donut Shop	Drugstore	Eastern European Restaurant	Hardware Store
1	M1C	Scarborough	Highland Creek, Rouge Hill, Port Union	43.784535	-79.160497	3.0	Bar	Yoga Studio	Discount Store	Dog Run	Doner Restaurant	Donut Shop	Drugstore	Dumpling Restaurant	Eastern European Restaurant	Field
2	M1E	Scarborough	Guildwood, Morningside, West Hill	43.763573	-79.188711	0.0	Pizza Place	Breakfast Spot	Rental Car Location	Intersection	Mexican Restaurant	Medical Center	Electronics Store	Spa	Eastern European Restaurant	Dumpling Restaurant
3	M1G	Scarborough	Woburn	43.770992	-79.216917	0.0	Coffee Shop	Korean Restaurant	Yoga Studio	Eastern European Restaurant	Discount Store	Dog Run	Doner Restaurant	Donut Shop	Drugstore	Dumpling Restaurant
4	M1H	Scarborough	Cedarbrae	43.773136	-79.239476	0.0	Hakka Restaurant	Athletics & Sports	Caribbean Restaurant	Thai Restaurant	Bakery	Bank	Fried Chicken Joint	Yoga Studio	Donut Shop	Dog Run

In [ ]:
toronto_merged[toronto_merged['Cluster Labels'].isnull()]

	PostalCode	Borough	Neighborhood	Latitude	Longitude	Cluster Labels	1st Most Common Venue	2nd Most Common Venue	3rd Most Common Venue	4th Most Common Venue	5th Most Common Venue	6th Most Common Venue	7th Most Common Venue	8th Most Common Venue	9th Most Common Venue	10th Most Common Venue
16	M1X	Scarborough	Upper Rouge	43.836125	-79.205636	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN
20	M2L	North York	Silver Hills, York Mills	43.757490	-79.374714	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN
21	M2M	North York	Newtonbrook, Willowdale	43.789053	-79.408493	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN
93	M9A	Etobicoke	Islington Avenue	43.667856	-79.532242	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN
98	M9N	York	Weston	43.706876	-79.518188	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN	NaN

Plot the clusters in the map

In [ ]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

toronto_merged_nonan = toronto_merged.dropna(subset=['Cluster Labels'])

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged_nonan['Latitude'], toronto_merged_nonan['Longitude'], toronto_merged_nonan['Neighborhood'], toronto_merged_nonan['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Examine clusters

Cluster 1

In [ ]:
toronto_merged_nonan.loc[toronto_merged_nonan['Cluster Labels'] == 0, toronto_merged_nonan.columns[[1] + list(range(5, toronto_merged_nonan.shape[1]))]]

	Borough	Cluster Labels	1st Most Common Venue	2nd Most Common Venue	3rd Most Common Venue	4th Most Common Venue	5th Most Common Venue	6th Most Common Venue	7th Most Common Venue	8th Most Common Venue	9th Most Common Venue	10th Most Common Venue
0	Scarborough	0.0	Fast Food Restaurant	Dumpling Restaurant	Diner	Discount Store	Dog Run	Doner Restaurant	Donut Shop	Drugstore	Eastern European Restaurant	Hardware Store
2	Scarborough	0.0	Pizza Place	Breakfast Spot	Rental Car Location	Intersection	Mexican Restaurant	Medical Center	Electronics Store	Spa	Eastern European Restaurant	Dumpling Restaurant
3	Scarborough	0.0	Coffee Shop	Korean Restaurant	Yoga Studio	Eastern European Restaurant	Discount Store	Dog Run	Doner Restaurant	Donut Shop	Drugstore	Dumpling Restaurant
4	Scarborough	0.0	Hakka Restaurant	Athletics & Sports	Caribbean Restaurant	Thai Restaurant	Bakery	Bank	Fried Chicken Joint	Yoga Studio	Donut Shop	Dog Run
6	Scarborough	0.0	Discount Store	Chinese Restaurant	Bus Station	Department Store	Coffee Shop	Convenience Store	Dog Run	Doner Restaurant	Donut Shop	Drugstore
7	Scarborough	0.0	Bus Line	Bakery	Fast Food Restaurant	Intersection	Bus Station	Soccer Field	Park	Ethiopian Restaurant	Empanada Restaurant	Electronics Store
8	Scarborough	0.0	Motel	Movie Theater	American Restaurant	Dim Sum Restaurant	Diner	Discount Store	Dog Run	Doner Restaurant	Donut Shop	Yoga Studio
9	Scarborough	0.0	College Stadium	General Entertainment	Skating Rink	Café	Drugstore	Diner	Discount Store	Dog Run	Doner Restaurant	Donut Shop
10	Scarborough	0.0	Indian Restaurant	Vietnamese Restaurant	Latin American Restaurant	Furniture / Home Store	Chinese Restaurant	Pet Store	Concert Hall	Dim Sum Restaurant	Event Space	Ethiopian Restaurant
11	Scarborough	0.0	Auto Garage	Middle Eastern Restaurant	Smoke Shop	Breakfast Spot	Sandwich Place	Shopping Mall	Donut Shop	Discount Store	Dog Run	Doner Restaurant
12	Scarborough	0.0	Lounge	Sandwich Place	Breakfast Spot	Skating Rink	Dumpling Restaurant	Dog Run	Doner Restaurant	Donut Shop	Drugstore	Electronics Store
13	Scarborough	0.0	Pizza Place	Thai Restaurant	Noodle House	Chinese Restaurant	Italian Restaurant	Fried Chicken Joint	Fast Food Restaurant	Pharmacy	Gift Shop	Electronics Store
15	Scarborough	0.0	Fast Food Restaurant	Chinese Restaurant	Pizza Place	Coffee Shop	Sandwich Place	Cosmetics Shop	American Restaurant	Breakfast Spot	Grocery Store	Pharmacy
17	North York	0.0	Mediterranean Restaurant	Dog Run	Pool	Golf Course	Dumpling Restaurant	Diner	Discount Store	Doner Restaurant	Donut Shop	Drugstore
18	North York	0.0	Clothing Store	Fast Food Restaurant	Coffee Shop	Restaurant	Tea Room	Bakery	Toy / Game Store	Jewelry Store	Burger Joint	Juice Bar
19	North York	0.0	Café	Japanese Restaurant	Bank	Chinese Restaurant	Yoga Studio	Dog Run	Doner Restaurant	Donut Shop	Drugstore	Dumpling Restaurant
22	North York	0.0	Ramen Restaurant	Restaurant	Café	Shopping Mall	Sandwich Place	Coffee Shop	Pizza Place	Arts & Crafts Store	Steakhouse	Ice Cream Shop
24	North York	0.0	Pizza Place	Coffee Shop	Butcher	Pharmacy	Empanada Restaurant	Electronics Store	Ethiopian Restaurant	Eastern European Restaurant	Dumpling Restaurant	Dessert Shop
26	North York	0.0	Café	Gym / Fitness Center	Caribbean Restaurant	Japanese Restaurant	Baseball Field	Drugstore	Discount Store	Dog Run	Doner Restaurant	Donut Shop
27	North York	0.0	Coffee Shop	Gym	Asian Restaurant	Beer Store	Bike Shop	Chinese Restaurant	Sporting Goods Shop	Japanese Restaurant	Clothing Store	General Entertainment
28	North York	0.0	Coffee Shop	Pharmacy	Frozen Yogurt Shop	Shopping Mall	Bridal Shop	Fast Food Restaurant	Sandwich Place	Diner	Bank	Supermarket
29	North York	0.0	Furniture / Home Store	Coffee Shop	Massage Studio	Bar	Dumpling Restaurant	Discount Store	Dog Run	Doner Restaurant	Donut Shop	Drugstore
31	North York	0.0	Shopping Mall	Grocery Store	Bank	Hotel	Dumpling Restaurant	Discount Store	Dog Run	Doner Restaurant	Donut Shop	Drugstore
32	North York	0.0	Food Truck	Home Service	Baseball Field	Yoga Studio	Dog Run	Doner Restaurant	Donut Shop	Drugstore	Dumpling Restaurant	Eastern European Restaurant
33	North York	0.0	Liquor Store	Athletics & Sports	Gym / Fitness Center	Discount Store	Grocery Store	Dumpling Restaurant	Dog Run	Doner Restaurant	Donut Shop	Drugstore
34	North York	0.0	Portuguese Restaurant	Coffee Shop	Hockey Arena	Intersection	Ethiopian Restaurant	Event Space	Empanada Restaurant	Falafel Restaurant	Electronics Store	Dim Sum Restaurant
35	East York	0.0	Pizza Place	Fast Food Restaurant	Athletics & Sports	Gastropub	Intersection	Pet Store	Pharmacy	Rock Climbing Spot	Café	Breakfast Spot
36	East York	0.0	Skating Rink	Park	Pharmacy	Video Store	Beer Store	Athletics & Sports	Asian Restaurant	Cosmetics Shop	Curling Ice	Donut Shop
37	East Toronto	0.0	Coffee Shop	Pub	Health Food Store	Neighborhood	Yoga Studio	Diner	Discount Store	Dog Run	Doner Restaurant	Donut Shop
38	East York	0.0	Sporting Goods Shop	Coffee Shop	Burger Joint	Sandwich Place	Supermarket	Sports Bar	Beer Store	Bike Shop	Smoothie Shop	Shopping Mall
65	Central Toronto	0.0	Sandwich Place	Café	Coffee Shop	Pizza Place	Pharmacy	American Restaurant	Burger Joint	Jewish Restaurant	Indian Restaurant	BBQ Joint
66	Downtown Toronto	0.0	Café	Bakery	Bar	Japanese Restaurant	Bookstore	Restaurant	Poutine Place	Beer Bar	Beer Store	Sandwich Place
67	Downtown Toronto	0.0	Café	Bar	Vegetarian  Vegan Restaurant	Bakery	Vietnamese Restaurant	Chinese Restaurant	Coffee Shop	Mexican Restaurant	Farmers Market	Dumpling Restaurant
68	Downtown Toronto	0.0	Airport Lounge	Airport Terminal	Airport Service	Plane	Harbor / Marina	Sculpture Garden	Boutique	Boat or Ferry	Airport Gate	Airport
69	Downtown Toronto	0.0	Coffee Shop	Restaurant	Café	Seafood Restaurant	Fast Food Restaurant	Cocktail Bar	Pub	Hotel	Cheese Shop	Farmers Market
70	Downtown Toronto	0.0	Coffee Shop	Café	Hotel	Steakhouse	Deli / Bodega	Seafood Restaurant	Bar	Bakery	Restaurant	Burger Joint
71	North York	0.0	Clothing Store	Vietnamese Restaurant	Miscellaneous Shop	Coffee Shop	Accessories Store	Event Space	Boutique	Womens Store	Furniture / Home Store	Ethiopian Restaurant
72	North York	0.0	Pizza Place	Pub	Asian Restaurant	Japanese Restaurant	Diner	Discount Store	Dog Run	Doner Restaurant	Donut Shop	Drugstore
73	York	0.0	Field	Dog Run	Hockey Arena	Trail	Dumpling Restaurant	Diner	Discount Store	Doner Restaurant	Donut Shop	Drugstore
75	Downtown Toronto	0.0	Grocery Store	Café	Park	Nightclub	Italian Restaurant	Diner	Baby Store	Athletics  Sports	Restaurant	Coffee Shop
76	West Toronto	0.0	Pharmacy	Bakery	Discount Store	Supermarket	Gym / Fitness Center	Music Venue	Pool	Middle Eastern Restaurant	Café	Brewery
77	West Toronto	0.0	Bar	Coffee Shop	Asian Restaurant	Mens Store	Boutique	Cocktail Bar	Pizza Place	Bakery	Café	New American Restaurant
78	West Toronto	0.0	Breakfast Spot	Café	Coffee Shop	Yoga Studio	Bar	Burrito Place	Restaurant	Caribbean Restaurant	Climbing Gym	Pet Store
80	York	0.0	Restaurant	Coffee Shop	Sandwich Place	Check Cashing Service	Turkish Restaurant	Yoga Studio	Donut Shop	Diner	Discount Store	Dog Run
81	York	0.0	Pizza Place	Caribbean Restaurant	Brewery	Bus Line	Drugstore	Discount Store	Dog Run	Doner Restaurant	Donut Shop	Dumpling Restaurant
82	West Toronto	0.0	Mexican Restaurant	Bar	Café	Gastropub	Diner	Bakery	Cajun / Creole Restaurant	Italian Restaurant	Furniture / Home Store	Speakeasy
83	West Toronto	0.0	Breakfast Spot	Gift Shop	Bookstore	Italian Restaurant	Bank	Bar	Restaurant	Dessert Shop	Movie Theater	Eastern European Restaurant
84	West Toronto	0.0	Coffee Shop	Pizza Place	Café	Italian Restaurant	Tea Room	Sushi Restaurant	Gourmet Shop	Bookstore	Food	Bar
85 Queen s Park	0.0	Coffee Shop	Gym	Japanese Restaurant	Burger Joint	Diner	Park	Smoothie Shop	Seafood Restaurant	Sandwich Place	Burrito Place
86 Mississauga	0.0	Hotel	Coffee Shop	Gym / Fitness Center	Mediterranean Restaurant	Fried Chicken Joint	Sandwich Place	Middle Eastern Restaurant	Burrito Place	American Restaurant	Drugstore
87 East Toronto	0.0	Yoga Studio	Garden	Brewery	Light Rail Station	Spa	Farmers Market	Fast Food Restaurant	Burrito Place	Restaurant	Recording Studio
88	Etobicoke	0.0	Café	Pharmacy	Bakery	Fried Chicken Joint	Fast Food Restaurant	Liquor Store	Mexican Restaurant	Pizza Place	Coffee Shop	Restaurant
89	Etobicoke	0.0	Pizza Place	Skating Rink	Dance Studio	Pharmacy	Coffee Shop	Pool	Pub	Sandwich Place	Gym	Airport Terminal
92	Etobicoke	0.0	Burger Joint	Wings Joint	Fast Food Restaurant	Gym	Grocery Store	Bakery	Supplement Shop	Convenience Store	Discount Store	Sandwich Place
94	Etobicoke	0.0	Bank	Golf Course	Yoga Studio	Dumpling Restaurant	Discount Store	Dog Run	Doner Restaurant	Donut Shop	Drugstore	Eastern European Restaurant
95	Etobicoke	0.0	Pizza Place	Beer Store	Convenience Store	Café	Shopping Plaza	Liquor Store	Pharmacy	Ethiopian Restaurant	Electronics Store	Eastern European Restaurant
96	North York	0.0	Pizza Place	Gym	Empanada Restaurant	Dumpling Restaurant	Diner	Discount Store	Dog Run	Doner Restaurant	Donut Shop	Drugstore
99	Etobicoke	0.0	Pizza Place	Intersection	Sandwich Place	Coffee Shop	Middle Eastern Restaurant	Chinese Restaurant	Doner Restaurant	Discount Store	Dog Run	Donut Shop
101	Etobicoke	0.0	Grocery Store	Pizza Place	Fried Chicken Joint	Coffee Shop	Sandwich Place	Fast Food Restaurant	Beer Store	Liquor Store	Pharmacy	Dance Studio
102	Etobicoke	0.0	Rental Car Location	Drugstore	Bar	Yoga Studio	Dim Sum Restaurant	Discount Store	Dog Run	Doner Restaurant	Donut Shop	Dumpling Restaurant

82 rows x 12 columns

Cluster 2

In [ ]:
toronto_merged_nonan.loc[toronto_merged_nonan['Cluster Labels'] == 1, toronto_merged_nonan.columns[[1] + list(range(5, toronto_merged_nonan.shape[1]))]]

	Borough	Cluster Labels	1st Most Common Venue	2nd Most Common Venue	3rd Most Common Venue	4th Most Common Venue	5th Most Common Venue	6th Most Common Venue	7th Most Common Venue	8th Most Common Venue	9th Most Common Venue	10th Most Common Venue
5	Scarborough	1.0	Playground	Yoga Studio	Dumpling Restaurant	Diner	Discount Store	Dog Run	Doner Restaurant	Donut Shop	Drugstore	Eastern European Restaurant
14	Scarborough	1.0	Playground	Park	Yoga Studio	Drugstore	Dim Sum Restaurant	Diner	Discount Store	Dog Run	Doner Restaurant	Donut Shop
48	Central Toronto	1.0	Playground	Gym	Yoga Studio	Dumpling Restaurant	Diner	Discount Store	Dog Run	Doner Restaurant	Donut Shop	Drugsto

Cluster 3

In [ ]:

toronto_merged_nonan.loc[toronto_merged_nonan['Cluster Labels'] == 2, toronto_merged_nonan.columns[[1] + list(range(5, toronto_merged_nonan.shape[1]))]]

	Borough	Cluster Labels	1st Most Common Venue	2nd Most Common Venue	3rd Most Common Venue	4th Most Common Venue	5th Most Common Venue	6th Most Common Venue	7th Most Common Venue	8th Most Common Venue	9th Most Common Venue	10th Most Common Venue
23	North York	2.0	Park	Bank	Yoga Studio	Dumpling Restaurant	Discount Store	Dog Run	Doner Restaurant	Donut Shop	Drugstore	Eastern European Restaurant
25	North York	2.0	Fast Food Restaurant	Park	Food & Drink Shop	Dumpling Restaurant	Diner	Discount Store	Dog Run	Doner Restaurant	Donut Shop	Drugstore
30	North York	2.0	Bus Stop	Park	Airport	Yoga Studio	Eastern European Restaurant	Dog Run	Doner Restaurant	Donut Shop	Drugstore	Dumpling Restaurant
40	East York	2.0	Park	Coffee Shop	Convenience Store	Yoga Studio	Dumpling Restaurant	Discount Store	Dog Run	Doner Restaurant	Donut Shop	Drugstore
44	Central Toronto	2.0	Gym / Fitness Center	Park	Bus Line	Swim School	Yoga Studio	Drugstore	Discount Store	Dog Run	Doner Restaurant	Donut Shop
50	Downtown Toronto	2.0	Park	Trail	Playground	Yoga Studio	Dim Sum Restaurant	Diner	Discount Store	Dog Run	Doner Restaurant	Donut Shop
74	York	2.0	Park	Fast Food Restaurant	Market	Pharmacy	Women's Store	Grocery Store	Department Store	Gym	Empanada Restaurant	Electronics Store
79	North York	2.0	Basketball Court	Park	Construction & Landscaping	Bakery	Electronics Store	Doner Restaurant	Donut Shop	Drugstore	Dumpling Restaurant	Eastern European Restaurant
90	Etobicoke	2.0	Park	River	Smoke Shop	Pool	Yoga Studio	Donut Shop	Dim Sum Restaurant	Diner	Discount Store	Dog Run
100	Etobicoke	2.0	Pizza Place	Park	Bus Line	Drugstore	Diner	Discount Store	Dog Run	Doner Restaurant	Donut Shop	Dumpling Restaurant

Cluster 4

In [ ]:

toronto_merged_nonan.loc[toronto_merged_nonan['Cluster Labels'] == 3, toronto_merged_nonan.columns[[1] + list(range(5, toronto_merged_nonan.shape[1]))]]

	Borough	Cluster Labels	1st Most Common Venue	2nd Most Common Venue	3rd Most Common Venue	4th Most Common Venue	5th Most Common Venue	6th Most Common Venue	7th Most Common Venue	8th Most Common Venue	9th Most Common Venue	10th Most Common Venue
1	Scarborough	3.0	Bar	Yoga Studio	Discount Store	Dog Run	Doner Restaurant	Donut Shop	Drugstore	Dumpling Restaurant	Eastern European Restaurant	Field

Cluster 5

In [ ]:
toronto_merged_nonan.loc[toronto_merged_nonan['Cluster Labels'] == 4, toronto_merged_nonan.columns[[1] + list(range(5, toronto_merged_nonan.shape[1]))]]

	Borough	Cluster Labels	1st Most Common Venue	2nd Most Common Venue	3rd Most Common Venue	4th Most Common Venue	5th Most Common Venue	6th Most Common Venue	7th Most Common Venue	8th Most Common Venue	9th Most Common Venue	10th Most Common Venue
91	Etobicoke	4.0	Baseball Field	Pool	Yoga Studio	Discount Store	Dog Run	Doner Restaurant	Donut Shop	Drugstore	Dumpling Restaurant	Eastern European Restaurant
97	North York	4.0	Baseball Field	Paper / Office Supplies Store	Yoga Studio	Discount Store	Dog Run	Doner Restaurant	Donut Shop	Drugstore	Dumpling Restaurant	Eastern European Restauran